In [1]:
# ==========================================
# SUPER CELL: CREATE APP + RUN TUNNEL
# ==========================================
import urllib
import time
import subprocess
import os

# --- STEP 1: WRITE THE APP CODE TO A FILE ---
print("📝 Creating app.py file...")

code = """
import streamlit as st
import pandas as pd
import pdfplumber
import re
import os
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util

# 1. CONFIGURATION & STYLING
st.set_page_config(page_title="AI Resume Matcher", layout="wide")

st.markdown(\"\"\"
<style>
    .stApp { background-color: #0E1117; }
    div.css-1r6slb0.e1tzin5v2 { background-color: #161B22; border: 1px solid #30363D; border-radius: 10px; padding: 20px; margin-bottom: 20px; }
    .match-circle { font-size: 24px; font-weight: bold; color: white; background-color: #21262D; border-radius: 50%; width: 60px; height: 60px; display: flex; align-items: center; justify-content: center; border: 3px solid; }
    .skill-badge-missing { background-color: rgba(255, 0, 0, 0.1); color: #FF6E6E; border: 1px solid #FF6E6E; padding: 4px 8px; border-radius: 15px; font-size: 0.85em; margin-right: 5px; margin-bottom: 5px; display: inline-block; }
    .match-badge-high { background-color: rgba(0, 255, 0, 0.1); color: #2EA043; border: 1px solid #2EA043; padding: 4px 8px; border-radius: 4px; }
    .match-badge-med { background-color: rgba(255, 165, 0, 0.1); color: #D29922; border: 1px solid #D29922; padding: 4px 8px; border-radius: 4px; }
    .match-badge-low { background-color: rgba(255, 0, 0, 0.1); color: #F85149; border: 1px solid #F85149; padding: 4px 8px; border-radius: 4px; }
</style>
\"\"\", unsafe_allow_html=True)

# 2. BACKEND LOGIC
@st.cache_resource
def load_model():
    return SentenceTransformer('all-MiniLM-L6-v2')

@st.cache_data
def load_jobs_data():
    dataset = load_dataset("jacob-hugging-face/job-descriptions", split="train[:300]")
    df = pd.DataFrame(dataset)
    if 'job_description' in df.columns:
        df = df.rename(columns={'job_description': 'Job Description', 'position_title': 'Job Title'})
    return df

model = load_model()
df_jobs = load_jobs_data()

def get_skill_taxonomies():
    tech_skills = {
        "python", "java", "c++", "c#", "javascript", "typescript", "ruby", "php", "swift", "kotlin",
        "react", "angular", "vue", "node.js", "django", "flask", "spring boot", "dotnet",
        "html", "css", "sql", "nosql", "mysql", "postgresql", "mongodb", "oracle",
        "aws", "azure", "gcp", "docker", "kubernetes", "jenkins", "terraform", "git", "github",
        "gitlab", "jira", "confluence", "tableau", "powerbi", "excel", "figma",
        "machine learning", "deep learning", "nlp", "tensorflow", "pytorch", "scikit-learn",
        "oop", "object oriented", "functional programming", "rest api", "graphql", "ci/cd",
        "devops", "agile", "scrum", "kanban", "sdlc", "data structures", "algorithms",
        "linux", "unix", "bash", "shell"
    }
    soft_skills = {
        "communication", "leadership", "teamwork", "problem solving", "critical thinking",
        "time management", "adaptability", "collaboration", "creativity", "emotional intelligence",
        "negotiation", "conflict resolution", "decision making", "mentoring", "presentation",
        "active listening", "flexibility", "work ethic", "detail oriented", "stakeholder management"
    }
    return tech_skills, soft_skills

TECH_SKILLS, SOFT_SKILLS = get_skill_taxonomies()
ALL_SKILLS = TECH_SKILLS.union(SOFT_SKILLS)

def get_implications():
    return {
        "oop": ["java", "c++", "c#", "python", "ruby"],
        "object oriented": ["java", "c++", "c#", "python", "ruby"],
        "cloud": ["aws", "azure", "gcp"],
        "frontend": ["react", "angular", "vue", "html", "css", "javascript"],
        "backend": ["node.js", "django", "spring", "flask"],
        "database": ["sql", "mysql", "postgresql", "mongodb", "oracle"],
        "ci/cd": ["jenkins", "gitlab", "github actions"],
        "devops": ["docker", "kubernetes", "jenkins", "terraform"]
    }
IMPLICATION_RULES = get_implications()

def extract_text_from_pdf(uploaded_file):
    text = ""
    try:
        with pdfplumber.open(uploaded_file) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + " "
    except Exception as e:
        st.error(f"Error reading PDF: {e}")
        return None
    return text.strip()

def analyze_split_gaps(resume_text, jd_text):
    resume_lower = resume_text.lower()
    jd_lower = jd_text.lower()
    missing_tech = []
    missing_soft = []

    def check_category(category_set, output_list):
        required = set()
        for skill in category_set:
            pattern = r'\\b' + re.escape(skill) + r'\\b'
            if re.search(pattern, jd_lower):
                required.add(skill)

        for skill in required:
            pattern = r'\\b' + re.escape(skill) + r'\\b'
            if not re.search(pattern, resume_lower):
                found_implied = False
                if skill in IMPLICATION_RULES:
                    for evidence in IMPLICATION_RULES[skill]:
                        evidence_pattern = r'\\b' + re.escape(evidence) + r'\\b'
                        if re.search(evidence_pattern, resume_lower):
                            found_implied = True
                            break
                if not found_implied:
                    output_list.append(skill.title())

    check_category(TECH_SKILLS, missing_tech)
    check_category(SOFT_SKILLS, missing_soft)
    return missing_tech, missing_soft

def calculate_keyword_score(resume_text, jd_text):
    jd_lower = jd_text.lower()
    resume_lower = resume_text.lower()
    required_skills = set()
    for skill in ALL_SKILLS:
        pattern = r'\\b' + re.escape(skill) + r'\\b'
        if re.search(pattern, jd_lower):
            required_skills.add(skill)

    if not required_skills: return 0.0

    matched_skills = 0
    for skill in required_skills:
        pattern = r'\\b' + re.escape(skill) + r'\\b'
        if re.search(pattern, resume_lower):
            matched_skills += 1
        elif skill in IMPLICATION_RULES:
            for evidence in IMPLICATION_RULES[skill]:
                evidence_pattern = r'\\b' + re.escape(evidence) + r'\\b'
                if re.search(evidence_pattern, resume_lower):
                    matched_skills += 1
                    break
    return (matched_skills / len(required_skills)) * 100

def match_resume_to_jobs_hybrid(resume_text, df):
    resume_embedding = model.encode(resume_text, convert_to_tensor=True)
    jd_embeddings = model.encode(df['Job Description'].tolist(), convert_to_tensor=True)
    cosine_scores = util.cos_sim(resume_embedding, jd_embeddings)[0]

    final_scores = []
    for i, jd_text in enumerate(df['Job Description']):
        semantic_score = float(cosine_scores[i]) * 100
        keyword_score = calculate_keyword_score(resume_text, jd_text)
        final_score = (semantic_score * 0.70) + (keyword_score * 0.30)
        final_scores.append(round(final_score, 1))

    df['Match Percentage'] = final_scores
    return df.sort_values(by='Match Percentage', ascending=False)

# 3. FRONTEND UI
col_header1, col_header2 = st.columns([3, 1])
with col_header1:
    st.title("Your Career Matches")
    st.markdown("Based on your resume analysis against **300 real-world job descriptions**.")

uploaded_file = st.file_uploader("Upload your Resume (PDF)", type=["pdf"])

if uploaded_file is not None:
    with st.spinner("Analyzing Resume and Matching Jobs..."):
        resume_text = extract_text_from_pdf(uploaded_file)

        if resume_text:
            results = match_resume_to_jobs_hybrid(resume_text, df_jobs)
            top_matches = results.head(6)

            st.markdown("---")
            for row_idx in range(2):
                cols = st.columns(3)
                for col_idx in range(3):
                    data_idx = row_idx * 3 + col_idx
                    if data_idx >= len(top_matches): break

                    row_data = top_matches.iloc[data_idx]
                    score = row_data['Match Percentage']
                    job_title = row_data['Job Title']
                    jd_text = row_data['Job Description']

                    tech_gaps, soft_gaps = analyze_split_gaps(resume_text, jd_text)
                    total_gaps = len(tech_gaps) + len(soft_gaps)

                    if score >= 75:
                        badge_class = "match-badge-high"
                        border_color = "#2EA043"
                        summary = "Your technical skills align very well with this role."
                    elif score >= 60:
                        badge_class = "match-badge-med"
                        border_color = "#D29922"
                        summary = "Good match. A few key skills would make you a top candidate."
                    else:
                        badge_class = "match-badge-low"
                        border_color = "#F85149"
                        summary = "Significant skill gaps found for this specific role requirement."

                    with cols[col_idx]:
                        with st.container():
                            c1, c2 = st.columns([3, 1])
                            with c1: st.subheader(f"📌 {job_title}")
                            with c2: st.markdown(f'<div class="match-circle" style="border-color: {border_color};">{int(score)}</div>', unsafe_allow_html=True)

                            st.markdown(f'<span class="{badge_class}">{score}% Match</span>', unsafe_allow_html=True)
                            st.markdown(f"<p style='color: #8B949E; font-size: 0.9em; margin-top: 10px;'>{summary}</p>", unsafe_allow_html=True)

                            st.markdown(f"##### ⚠️ Skills to Learn ({total_gaps})")
                            badge_html = ""
                            for skill in tech_gaps[:3]:
                                badge_html += f'<span class="skill-badge-missing">{skill}</span>'

                            if not tech_gaps and not soft_gaps:
                                st.markdown("<small style='color: green;'>Perfect technical match!</small>", unsafe_allow_html=True)
                            else:
                                st.markdown(badge_html, unsafe_allow_html=True)

                            with st.expander("View All Skills & Gaps"):
                                st.write("**Job Description Snippet:**")
                                st.caption(jd_text[:200] + "...")
                                st.write("**Missing Technical Skills:**")
                                st.write(", ".join(tech_gaps) if tech_gaps else "None")
                                st.write("**Missing Soft Skills:**")
                                st.write(", ".join(soft_gaps) if soft_gaps else "None")
                        st.markdown("---")
else:
    st.info("👆 Please upload your resume PDF to see your career matches.")
"""
with open("app.py", "w") as f:
    f.write(code)

# --- STEP 2: INSTALL DEPENDENCIES & TUNNEL ---
print("🚀 Installing Libraries & Cloudflare Tunnel...")
!pip install streamlit sentence-transformers pandas pdfplumber datasets scikit-learn -q
!wget -q -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

# --- STEP 3: RUN THE APP ---
print("⚡ Starting Streamlit App (This takes ~60 seconds to load models)...")
!pkill -f streamlit
!pkill -f cloudflared

# Run Streamlit in background and log to a file
with open("streamlit_logs.txt", "w") as log_file:
    process = subprocess.Popen(["streamlit", "run", "app.py"], stdout=log_file, stderr=log_file)

# Run Tunnel
!nohup ./cloudflared-linux-amd64 tunnel --url http://localhost:8501 > tunnel_logs.txt 2>&1 &

time.sleep(5)
print("--------------------------------------------------")
print("👇 CLICK THIS LINK TO OPEN YOUR APP 👇")
!grep -o 'https://.*\.trycloudflare.com' tunnel_logs.txt | head -n 1
print("--------------------------------------------------")

print("⏳ Waiting for app to load models... (Watch the logs below)")
print("PLEASE WAIT until you see 'Model loaded' before clicking the link!")
!tail -f streamlit_logs.txt

📝 Creating app.py file...
🚀 Installing Libraries & Cloudflare Tunnel...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 56.3 MB/s eta 0:00:00
⚡ Starting Streamlit App (This takes ~60 seconds to load models)...
--------------------------------------------------
👇 CLICK THIS LINK TO OPEN YOUR APP 👇
https://menus-fusion-enemies-ranges.trycloudflare.com
--------------------------------------------------
⏳ Waiting for app to load models... (Watch the logs below)
PLEASE WAIT until you see 'Model loaded' before clicking the l